In [1]:
import math
import random
import numpy as np
from Iogurt import *
import os
import time


import matplotlib.pyplot as plt
from ase.data import atomic_numbers, reference_states #Convenient atomic information from reputable sources
from ase.md.andersen import Andersen #The NVT thermostat used in LoDiS
from ase.io import read, write #Input/Output manager for atoms
from ase.build import fcc111, fcc100
from asap3 import * #Just throw everything from asap3
from asap3 import Atoms, EMT, units, Trajectory #You can get specific utility from each of these sub-modules
from asap3.md.langevin import Langevin
from asap3.md.velocitydistribution import *

#Below are specific paramters for computing inter-atomic forces according to the RGL formula 
from GuptaParameters import AuCu_parameters as AuCu_Gupta
from GuptaParameters import AuPt_parameters as AuPt_Gupta
from GuptaParameters import Au_parameters as Au_Gupta

In [8]:
#In this box we can compute the files for the configurations 1 to 9:

#We are going to compute several configurations to observe how the energy changes depending on the configuration.
#All of this configurations will be composed by NP with 147 to 586 atoms. 

###################################################################################################################
#In this area we introduce our parameters for the next simulations

#Sizes of the box
lx=100
ly=100
lz=100

#Number of proposed x-coordinates for each deposition
N=5000

#Folder with the .xyz files
d='/home/tonidemarti/Documents/Antonio_de_Marti/Data/Au147586/'

#Number of configurations
conf=1  

#Total number of NP we want in our foams
M=15

#Note: if we wanted to have a random number of NP in each configuration we would have to define M inside the following code as it is padded.


###################################################################################################################
#Here we write the files that we will make use of later

#We begin by opening all of the files and writing a quick first introductory row:

with open('Configurations.dat', 'w') as f:
    with open('Centers of mass of NP.dat', 'w') as g:
        g.write('Centers of mass: \n')
        g.write('xf'+'  \t\t\t\t  '+'yf'+'  \t\t\t  '+'zf'+'  \t\t  '+'NP number'+' \n')
        with open('NPs used in the configurations.dat','w') as t:
            t.write('NP used  \t  Number of atoms within the NP \n')
            with open('Adjacent matrix.dat','w') as ff:
                ff.write('Configuration number  \t  NP number  \t\t Number of adjacent NP \n')
                with open ('Energy costs.dat','w') as gg:
                    gg.write('Configuration  \t\t  Energy system  \t\t  Energy Bulk  \t\t Energy cost per NP \t Energy cost per atom \n')
                    
                    
                    
#We can now start a loop which will repeat the process of constructing the nanofoam for "conf" number of times.                    
#We introduce the information for our configuration file, which will be the one with the most information.
#I also print the number of configuration to keep track of the evolution of the program while it is running.

                    for i in range(0,conf):
                        print('Configuration number: ',i+1)
                        f.write('Configuration number: '+ str(i+1))

#Here you can unpad the M's if you wanted to compute the nanofoam of a rnadom number of NP from 0 to 550.

                        #M=int(550*random.random())
                        #print('Number of NP: ', M)
        
#We now execute the Llivia function, and obtain the meaningful values which allow us to write the files.

                        V_ratio, radius, xf, yf, zf, file, nfile, matrix, number = llivia(d,M,lx,ly,lz,N)
    
#First we use the file output from the case c function to write the location of all the NP's used in the configuration.
#Notice at the end that whenever we finish the ongoing configuration and jump to the next one we will add two
#white lines to know the files correspond to another configuration.

                        for j in range (0,len(file)):
                            t.write(str(j)+ '\t' + file[j]+ '\t' +str(nfile[j]) + '\n')
                        t.write('\n\n')
                
#We then proceed to compute the adjacent matrix file. We do so by constructing a list as long as the number of 
#NP com. For every slot in the list we compute a summation of all the values in its raw. Since non-adjacent NP's
#have a value of 0 in the adjacent matrix they will not contribute while adjacent NP will contribute with a value
#of 1. Doing the summation of the rows for each slot of the list will derive in a list of adjacent NP's.
#Lastly, we can obtain the average number of adjacent NP's by dividing the summation of the list adj over the number
#of NP's M. The standard deviation is also obtained and both values are introduced on the last row of the configuration.
                        
                        adj=[0]*len(xf)
                        for j in range (0,len(xf)):
                            adjacent=0
                            for k in range(0,len(xf)):
                                adj[j]=adj[j]+matrix[j,k]
                            ff.write('\t'+ str(i)+'  \t\t\t  '+str(j)+'  \t\t\t  '+str(int(adj[j]))+'\n')
                        
                        avadj=sum(adj)/M
                        deviation=0
                        for j in range (0,len(adj)):
                            deviation=deviation+(abs(avadj-adj[j]))/M

                        ff.write('The average number of adjacent NP is :'+ str(avadj)+ '  +/-  '+str(deviation)+'\n')
                        ff.write('\n\n')

#The average size(number of atoms) per particle is obtained by dividing the total number of gold atoms (sum(nfile))
#over the total number of nanoparticles. The standard deviation can also be included.

                        avsize=sum(nfile)/M
                        deviation=0
                        for j in range(0,len(nfile)):
                            deviation=deviation+abs(avsize-nfile[j])/(len(nfile))

#We now proceed to compute the energy of the configuration through an RGL calculator.

                        RGL_Calc = RGL(Au_Gupta) #The LoDiS type calculator
                        NP= read('Nanofoam.xyz')
                        NP.set_cell((lx,ly,lz))
                        NP.set_calculator(RGL_Calc)
                        ekin = NP.get_kinetic_energy()
                        epot = NP.get_potential_energy()
                        a=ekin+epot
                        
#Once we have the energy of the nanofoam we compute the energy of a hypothetical initial bulk. This initial bulk
#consists in the same number of gold atoms that form the nanofoam arranged forming a bulk made of yz-layers of thickness
#of 2 Au atoms.

                        print('Number: ', number)
                        print(flour(lx,ly,lz,number))
                        BLCK= read('Bulk.xyz')
                        BLCK.set_cell((lx,ly,lz))
                        BLCK.set_calculator(RGL_Calc)
                        ekinb = BLCK.get_kinetic_energy()
                        epotb = BLCK.get_potential_energy()
                        b=ekinb+epotb
                        
#We can now compare the energy of the nanofoam with that of the slab and have a meaningful result of the Energy cost
                        gg.write(str(i+1)+'  \t\t\t\t  '+str(a)+'  \t\t  '+str(b)+'  \t\t  '+str((a-b)/M) +'  \t  '+str((a-b)/number)+'\n') 
                        
#Lastly, we proceed to continue writing the configuration file. We write some meaningful data to introduce the
#computed configuration:

                        f.write('Etot (RGL):  '+ str(a)+' \t\t '+'Number of NP: ' +str(M)+'\n'+'Average NP size: '+str(avsize)+'\t Standard Size deviation: '+str(deviation)+ '\n')
                        f.write('Element \t x  \t\t\t  y  \t\t\t  z  \t\t  NP number \n')

#Afterwards we load the data from the .xyz file created by the case c function and introduce it to the configuration file.

                        b=np.loadtxt('Nanofoam.xyz', usecols=(1,2,3,4), skiprows=2)
                        x=b[:,0]
                        y=b[:,1]
                        z=b[:,2]
                        name=b[:,3]
                        for j in range (0,len(x)):
                            if name[j]>0:
                                f.write('Au \t'+str(x[j])+'  \t  '+str(y[j])+'  \t  '+str(z[j])+'  \t  '+str(int(name[j]))+' \n')
                        f.write('\n  \n') 

#We finish by writing the file including the positions of all the NP's com.

                        for j in range (0,len(xf)):
                            g.write(str(xf[j])+'  \t  '+str(yf[j])+'  \t  '+str(zf[j])+'  \t  '+str(j+1)+' \n')
                        g.write('\n  \n')
                        


Configuration number:  1
Check:  1
Recursions:  1
Check:  2
Recursions:  1
Check:  3
Recursions:  1
Check:  4
Recursions:  1
Check:  5
Recursions:  1
Check:  6
Recursions:  1
Check:  7
Recursions:  1
Check:  8
Recursions:  1
Check:  9
Recursions:  1
Check:  10
Recursions:  1
Check:  11
Recursions:  1
Recursions:  2
Recursions:  3
Recursions:  4
Recursions:  5
Recursions:  6
Check:  12
Recursions:  1
Recursions:  2
Check:  13
Recursions:  1
Check:  14
Recursions:  1
Recursions:  2
Check:  15
Recursions:  1
Recursions:  2
Recursions:  3
Recursions:  4
Recursions:  5
Occ Volume 133042.8549398244
Available Volume 1000000
V_ratio:    0.1330428549398244
Low density system
Number:  6155
None


In [10]:
#In this box we can compute the files for the configurations 10 to 14:

#Note, the whole box is almost identical to the previous. The only change ist the introduction of the "probs"
#parameter and the usage of case_d instead of case_c.

###################################################################################################################
#In this area we introduce our parameters for the next simulations


#Sizes of the box
lx=350
ly=350
lz=350

#Number of proposed x-coordinates for each deposition
N=5000

#Folder with the .xyz files
d='/home/tonidemarti/Documents/Antonio_de_Marti/Data/Case B/'

#Number of configurations
conf=4

#Probability list. 
probs=[445/894, 449/894]


#Total number of NP we want in our nanofoam
M=15


#Note: if we wanted to have a random number of NP in each configuration we would have to define M inside the following code as it is padded.



###################################################################################################################
#Here we write the files that we will make use of later

#We begin by opening all of the files and writing a quick first introductory row:

with open('Configurations.dat', 'w') as f:
    with open('Centers of mass of NP.dat', 'w') as g:
        g.write('Centers of mass: \n')
        g.write('xf'+'  \t\t\t\t  '+'yf'+'  \t\t\t  '+'zf'+'  \t\t  '+'NP number'+' \n')
        with open('NPs used in the configurations','w') as t:
            t.write('NP used  \t  Number of atoms within the NP \n')
            with open('Adjacent matrix.dat','w') as ff:
                ff.write('Configuration number  \t  NP number  \t\t Number of adjacent NP \n')
                with open ('Energy costs','w') as gg:
                    gg.write('Configuration  \t\t  Energy system  \t\t  Energy Bulk  \t\t Energy cost per NP \t Energy cost per atom \n')
                    
#We can now start a loop which will repeat the process of constructing the nanofoam for "conf" number of times.                    
#We introduce the information for our configuration file, which will be the one with the most information.
#I also print the number of configuration to keep track of the evolution of the program while it is running.


                    for i in range(0,conf):
                        print('Configuration number: ',i+1)
                        f.write('Configuration number: '+ str(i+1))
                
#Here you can unpad the M's if you wanted to compute the nanofoam of a rnadom number of NP from 0 to 550.

                        #M=int(550*random.random())
                        #print('Number of NP: ', M)
        
#We now execute the case_d function, and obtain the meaningful values which allow us to write the files.

                        V_ratio, radius, xf, yf, zf, file, nfile, matrix, number = Forn(d,M,lx,ly,lz,N,probs)
    
#First we use the file output from the case c function to write the location of all the NP's used in the configuration.
#Notice at the end that whenever we finish the ongoing configuration and jump to the next one we will add two
#white lines to know the files correspond to another configuration.

                        for j in range (0,len(file)):
                            t.write(str(j)+ '\t' + file[j]+ '\t' +str(nfile[j]) + '\n')
            
#We then proceed to compute the adjacent matrix file. We do so by constructing a list as long as the number of 
#NP com. For every slot in the list we compute a summation of all the values in its raw. Since non-adjacent NP's
#have a value of 0 in the adjacent matrix they will not contribute while adjacent NP will contribute with a value
#of 1. Doing the summation of the rows for each slot of the list will derive in a list of adjacent NP's.
#Lastly, we can obtain the average number of adjacent NP's by dividing the summation of the list adj over the number
#of NP's M. The standard deviation is also obtained and both values are introduced on the last row of the configuration.
                        
                        adj=[0]*len(xf)
                        for j in range (0,len(xf)):
                            adjacent=0
                            for k in range(0,len(xf)):
                                adj[j]=adj[j]+matrix[j,k]
                            ff.write('\t'+ str(i)+'  \t\t\t  '+str(j)+'  \t\t\t  '+str(int(adj[j]))+'\n')

                        avadj=sum(adj)/M
                        deviation=0
                        for j in range (0,len(adj)):
                            deviation=deviation+(abs(avadj-adj[j]))/M

                        ff.write('The average number of adjacent NP is :'+ str(avadj)+ '  +/-  '+str(deviation)+'\n')
                        ff.write('\n\n')

#The average size(number of atoms) per particle is obtained by dividing the total number of gold atoms (sum(nfile))
#over the total number of nanoparticles. The standard deviation can also be included.
                        
                        
                        avsize=sum(nfile)/M
                        deviation=0
                        for j in range(0,len(nfile)):
                            deviation=deviation+abs(avsize-nfile[j])/(len(nfile))

#We now proceed to compute the energy of the configuration through an RGL calculator.

                        RGL_Calc = RGL(Au_Gupta) #The LoDiS type calculator
                        NP= read('Nanofoam.xyz')
                        NP.set_cell((lx,ly,lz))
                        NP.set_calculator(RGL_Calc)
                        ekin = NP.get_kinetic_energy()
                        epot = NP.get_potential_energy()
                        a=ekin+epot

#Once we have the energy of the nanofoam we compute the energy of a hypothetical initial bulk. This initial bulk
#consists in the same number of gold atoms that form the nanofoam arranged forming a bulk made of yz-layers of thickness
#of 2 Au atoms.
                        
                        print(flour(lx,ly,lz,number))
                        BLCK= read('Bulk.xyz')
                        BLCK.set_cell((lx,ly,lz))
                        BLCK.set_calculator(RGL_Calc)
                        ekinb = BLCK.get_kinetic_energy()
                        epotb = BLCK.get_potential_energy()
                        b=ekinb+epotb

#We can now compare the energy of the nanofoam with that of the bulk and have a meaningful result of the Energy cost
                
                        gg.write(str(i+1)+'  \t\t\t\t  '+str(a)+'  \t\t  '+str(b)+'  \t\t  '+str((a-b)/M)+'  \t\t  ' +str((a-b)/number)+'\n') 
           
#Lastly, we proceed to continue writing the configuration file. We write some meaningful data to introduce the
#computed configuration:
            
                        f.write('Etot (RGL):  '+ str(a)+' \t\t '+'Number of NP: ' +str(M)+'\n'+'Average NP size: '+str(avsize)+'\t Standard Size deviation: '+str(deviation)+ '\n')
                        f.write('Element \t x  \t\t\t  y  \t\t\t  z  \t\t  NP number \n')
            
#Afterwards we load the data from the .xyz file created by the case c function and introduce it to the configuration file.

                        b=np.loadtxt('Nanofoam.xyz', usecols=(1,2,3,4), skiprows=2)
                        x=b[:,0]
                        y=b[:,1]
                        z=b[:,2]
                        name=b[:,3]
                        for j in range (0,len(x)):
                            if name[j]>0:
                                f.write('Au \t'+str(x[j])+'  \t  '+str(y[j])+'  \t  '+str(z[j])+'  \t  '+str(int(name[j]))+' \n')
                        
#We finish by writing the file including the positions of all the NP's com.

                        for j in range (0,len(xf)):
                            g.write(str(xf[j])+'  \t  '+str(yf[j])+'  \t  '+str(zf[j])+'  \t  '+str(j+1)+' \n')
                        f.write('\n  \n')
                        g.write('\n  \n')
                        t.write('\n\n')

Configuration number:  1
Check:  1
Recursions:  1
Check:  2
Recursions:  1
Check:  3
Recursions:  1
Check:  4
Recursions:  1
Check:  5
Recursions:  1
Check:  6
Recursions:  1
Check:  7
Recursions:  1
Check:  8
Recursions:  1
Check:  9
Recursions:  1
Check:  10
Recursions:  1
Check:  11
Recursions:  1
Check:  12
Recursions:  1
Check:  13
Recursions:  1
Check:  14
Recursions:  1
Check:  15
Recursions:  1
Occ Volume 40531.41784279558
Available Volume 42875000
V_ratio:    0.0009453391916687015
Low density system
None
Configuration number:  2
Check:  1
Recursions:  1
Check:  2
Recursions:  1
Check:  3
Recursions:  1
Check:  4
Recursions:  1
Check:  5
Recursions:  1
Check:  6
Recursions:  1
Check:  7
Recursions:  1
Check:  8
Recursions:  1
Check:  9
Recursions:  1
Check:  10
Recursions:  1
Check:  11
Recursions:  1
Check:  12
Recursions:  1
Check:  13
Recursions:  1
Check:  14
Recursions:  1
Check:  15
Recursions:  1
Occ Volume 39631.86120789093
Available Volume 42875000
V_ratio:    0.000924